In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#!pip install datasets transformers
#transformers==4.9.2
#datasets==1.11.0

In [3]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [4]:
from datasets import load_dataset
from datasets import load_metric
task = 'sst2'
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

/tmp/ipykernel_34/2385546831.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', actual_task)
/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [5]:
import datasets
import random
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
    
show_random_elements(dataset["train"])

,sentence,label,idx
0,the ingenuity that parker displays in freshening the play is almost in a class with that of wilde himself .,positive,9739
1,"of the most creative , energetic and original comedies to hit the screen in years",positive,34084
2,"there are laughs aplenty , and , as a bonus , viewers do n't have to worry about being subjected to farts , urine , feces , semen , or any of the other foul substances that have overrun modern-day comedies .",positive,51750
3,"'s coherent , well shot , and",positive,63219
4,graphic treatment,positive,30691
5,cruel and,negative,51881
6,rare documentary,positive,26725
7,a complete wash,negative,65931
8,"( less a movie than ) an appalling , odoriferous thing ... so rotten in almost every single facet of production that you 'll want to crawl up your own *** in embarrassment .",negative,13985
9,"and , thanks to the presence of ` the king , ' it also rocks .",positive,1904


# Data Preparation

* Obtain a tokenizer **corresponding to the pre-trained model** one by one.

* When using the specified model checkpoint's corresponding tokenizer, the vocabulary required by the model, to be precise, is downloaded, which is the tokens vocabulary.

In [6]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Example of Tokenize

In [7]:
batch_sentences = ["Hello I'm a single sentence",
                   "And another sentence",
                   "And the very very last one"]
encoded_inputs = tokenizer(batch_sentences)
print(encoded_inputs)

{'input_ids': [[101, 7592, 1045, 1005, 1049, 1037, 2309, 6251, 102], [101, 1998, 2178, 6251, 102], [101, 1998, 1996, 2200, 2200, 2197, 2028, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}


* input_ids: This represents the input text converted into corresponding token IDs. In this example, each input text is converted into a sequence of token IDs. For instance, [101, 7592, 1045, 1005, 1049, 1037, 2309, 6251, 102] is the token IDs for the first input text, where 101 is the ID for the [CLS] token, 102 is the ID for the [SEP] token, and the numbers in between are the IDs for individual words or subwords.

* token_type_ids: This indicates which sentence each token belongs to. In BERT, each input text is split into two sentences (possibly just one sentence). In this example, the length of each list matches the length of the corresponding input_ids, indicating which sentence each token belongs to. For this task, all sentences belong to the same segment, so all values are 0.

* attention_mask: This is a mask indicating which tokens are real and which are padding tokens. 1 indicates a real token, and 0 indicates a padding token. In this example, all tokens are real, so all values are 1.

In [8]:
#for a sentence
encoded_input = tokenizer("How old are you?", "I'm 6 years old")
print(encoded_input)

{'input_ids': [101, 2129, 2214, 2024, 2017, 1029, 102, 1045, 1005, 1049, 1020, 2086, 2214, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


If you have trained your own tokenizers, you can add a method to handle pairs of sentences as follows:

In [9]:
# from tokenizers.processors import TemplateProcessing

# tokenizer.post_processor = TemplateProcessing(
#     single="[CLS] $A [SEP]",
#     pair="[CLS] $A [SEP] $B:1 [SEP]:1",
#     special_tokens=[
#         ("[CLS]", tokenizer.token_to_id("[CLS]")),
#         ("[SEP]", tokenizer.token_to_id("[SEP]")),
#     ],
# )

# tokenizer.enable_truncation(max_length=512)
# tokenizer.save("data/tokenizer-wiki.json")

For different data and their corresponding data formats, in order to preprocess our data, define the following dict to separately use the tokenizer to handle cases where the input is a single sentence or a sentence pair.

In [10]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [11]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

We have already demonstrated the tokenizer processing a small batch. The dataset class can directly retrieve the corresponding index sentences 1 and 2, so **the above preprocessing function can handle both single samples and multiple samples.** If the input is multiple samples, the output is a list:

In [12]:
sentence1_key, sentence2_key = task_to_keys[task]
print(dataset['train'][:5])
preprocess_function(dataset['train'][:5])


{'sentence': ['hide new secretions from the parental units ', 'contains no wit , only labored gags ', 'that loves its characters and communicates something rather beautiful about human nature ', 'remains utterly satisfied to remain the same throughout ', 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up '], 'label': [0, 0, 1, 0, 0], 'idx': [0, 1, 2, 3, 4]}


{'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102], [101, 3397, 2053, 15966, 1010, 2069, 4450, 2098, 18201, 2015, 102], [101, 2008, 7459, 2049, 3494, 1998, 10639, 2015, 2242, 2738, 3376, 2055, 2529, 3267, 102], [101, 3464, 12580, 8510, 2000, 3961, 1996, 2168, 2802, 102], [101, 2006, 1996, 5409, 7195, 1011, 1997, 1011, 1996, 1011, 11265, 17811, 18856, 17322, 2015, 1996, 16587, 2071, 2852, 24225, 2039, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

Next, use the **map function** to preprocess all samples in the **three sample sets in the datasets**, and apply the preprocessing function prepare_train_features to all samples.

In [13]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

# Finetuning Pre-trained Models

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model_checkpoint = "distilbert-base-uncased"
num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

2024-05-06 08:26:06.079175: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 08:26:06.079288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 08:26:06.204288: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In order to obtain a Trainer training tool, we also need TrainingArguments for the configuration/settings of training. This training setup contains all the properties that define the training process.

In [15]:
batch_size = 16
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"

args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name, 
)

Another important thing, we need to choose a suitable evaluation metric to guide the model fine-tuning. We use the 🤗 Datasets library to load the evaluation metric calculation library load_metric. metic is an instance of datasets.Metric:

In [16]:
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)
#{'matthews_correlation': 0.1513518081969605}

{'accuracy': 0.46875}

Define the evaluation method for each task in Trainer as compute_metrics.

In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [18]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [19]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.180700,0.294788,0.905963
2,0.136500,0.384246,0.901376
3,0.092200,0.395704,0.902523
4,0.070700,0.445318,0.909404
5,0.043800,0.490704,0.907110


TrainOutput(global_step=21050, training_loss=0.11176590635204542, metrics={'train_runtime': 1025.8321, 'train_samples_per_second': 328.265, 'train_steps_per_second': 20.52, 'total_flos': 3067639398189732.0, 'train_loss': 0.11176590635204542, 'epoch': 5.0})

In [20]:
trainer.evaluate()

{'eval_loss': 0.4453180730342865,
 'eval_accuracy': 0.9094036697247706,
 'eval_runtime': 0.9142,
 'eval_samples_per_second': 953.8,
 'eval_steps_per_second': 60.159,
 'epoch': 5.0}

In [23]:
trainer.save_model("/kaggle/working/param")